In [1]:
pip install -q wikipedia d

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Vector database\nSummary: A vector database management system (VDBMS) or simply vector database or vector store is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In t'

# Reasoning and Acting (React)

## Creating a ReAct Agent

In [4]:
#pip install langchainhub -q

In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Answer the following questions in GERMAN as best uou can.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name = 'Python REPL',
    func = python_repl.run,
    description = 'Useful when you need to use Python to answer a question. You Should input Python code.'
)

# 2. Wikipedia Tool (for searching wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on wikipedia'
)
# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool cant provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations= 10
)

In [7]:
question = 'Generate the first 20 numbers in the Fibonacci series'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to generate the Fibonacci series in German.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci_series = [fibonacci(i) for i in range(20)]
fibonacci_series
```I have the numbers in the Fibonacci series.
Final Answer: Die ersten 20 Zahlen in der Fibonacci-Folge sind 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [8]:
question = 'Who is the current prime minister of the UK?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I need to find out who the current prime minister of the UK is in German.
Action: DuckDuckGo Search
Action Input: "Aktueller Premierminister des Vereinigten Königreichs"Der derzeitige Premierminister ist Rishi Sunak, der im Februar 2020 ernannt wurde. Er ist der erste britische Premierminister mit indischen Wurzeln. Außerdem ist er der jüngste Amtsinhaber seit dem Amtsantritt von David Cameron im Jahr 2010. Wie heißt der aktuelle Britische Premierminister? Nur fünf Tage nach dem Rücktritt (am 20.10.2022) seiner Vorgängerin Liz Truss, ernannte König Charles III. Rishi Sunak (*12.5.1980) am 25. Oktober 2022 zum 79. Britische Premierminister. 03.07.2023, 21:30 Uhr Von: Joshua Schößler Die Flagge des Vereinigten Königreichs, auch „Union Jack" genannt. © Dominic Lipinski/dpa Das Vereinigte Königreich ist einer der bevölkerungsreichsten Staaten Europas. 2020 verließ das Land die EU. Ein Überblick über dessen Geschichte und politisches System. Der aktue

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [9]:
print(output['input'])


Answer the following questions in GERMAN as best uou can.
Questions: Who is the current prime minister of the UK?



In [10]:
output['output']

'Der aktuelle Premierminister des Vereinigten Königreichs ist Boris Johnson.'

In [11]:
question = 'Tell me about Napolean bonaparte?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I need to find information about Napoleon Bonaparte in German.
Action: Wikipedia
Action Input: Napoleon Bonaparte German WikipediaPage: Napoleon II
Summary: Napoleon II (Napoléon François Joseph Charles Bonaparte; 20 March 1811 – 22 July 1832) was the disputed Emperor of the French for a few weeks in 1815. He was the son of Emperor Napoleon I and Empress Marie Louise, daughter of Emperor Francis I of Austria. Napoleon II had been Prince Imperial of France and King of Rome since birth. After the fall of his father, he lived the rest of his life in Vienna and was known in the Austrian court as Franz, Duke of Reichstadt  for his adult life (from the German version of his second given name, along with a title his grandfather granted him in 1818).  He was posthumously given the nickname L'Aiglon ("the Eaglet") after the popular Edmond Rostand play, L'Aiglon.
When Napoleon I tried to abdicate on 4 April 1814, he said that his son would rule as emperor.

# Vector Databases

In [12]:
# Pinecone is a vector database that is designed for storing a query of high dimensional
# vectors. It provides fast and efficient semantic search over vector embeddings by integrating open ai
# open ai LLMs, we fine deep learning capability for embedding generation with efficient vector search
# storage and retrieval .

# Vector databases are a new type of database, designed to store and query unstructured data.
# Unstructured data is data that does not have a fixed schema, such as text, images and audio.

# Pipeline for Vector Databases
# Vector databases use a combination of different optimized algorithms that all participate in
# Approximate Nearest Neighbor (ANN) search.

# Steps 1> Embedding, 2> Indexing, 3> Querying




In [13]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [14]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install --upgrade -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip show pinecone-client

Name: pinecone-client
Version: 3.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [21]:
from pinecone import Pinecone
pc = Pinecone()
#pc = Pinecone(api_key='d5fa5468-8b37-4add-a187-42f0e7c3c675')
pc.list_indexes()

{'indexes': []}

In [19]:
pip install -q tqdm

Note: you may need to restart the kernel to use updated packages.


In [20]:
from tqdm.autonotebook import tqdm

In [23]:
pc.list_indexes()[0]

IndexError: list index out of range

In [25]:
pc.describe_index('langchain')

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': 'a9726d006ab4de2df36ea290be74fac6', 'Date': 'Wed, 28 Feb 2024 05:13:52 GMT', 'Server': 'Google Frontend', 'Content-Length': '84', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource langchain not found"},"status":404}


In [27]:
from pinecone import PodSpec
index_name = 'langchain'

if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=PodSpec(
        environment='gcp-starter'
    )
    )
    
    print('Index Created!')
else:
    print(f'Index {index_name} already exists!')
    


Creating index langchain
Index Created!


In [28]:
index_name = 'lanchain'
if index_name in pc.list_indexes().names():
    print(f')